In [38]:

!pip install weaviate-client langchain tiktoken pypdf rapidocr-onnxruntime


In [39]:
WEAVIATE_CLUSTER="https://8zykxy8qfi0qia5wnviiq.c0.asia-southeast1.gcp.weaviate.cloud"
WEAVIATE_API_KEY="EZzXTDze5QJJ5uerQSn5fZfKZF9DX6Ru4NWL"

In [40]:
from langchain.vectorstores import Weaviate
import weaviate

WEAVIATE_URL = WEAVIATE_CLUSTER
WEAVIATE_API_KEY = WEAVIATE_API_KEY

client = weaviate.Client(
    url=WEAVIATE_URL, auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY)
)

In [41]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [42]:
!pip install sentence-transformers

In [43]:
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
#model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
  model_name=embedding_model_name,
  #model_kwargs=model_kwargs
)

In [44]:
!pip install langchain-community

loading pdf


In [81]:
from langchain.document_loaders import PyPDFLoader

# List of PDF file paths
pdf_files = ["/content/lora.pdf", "/content/qlora.pdf", "/content/rag.pdf"]

# Load all PDFs
pages = []
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    pages.extend(loader.load())

In [45]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/rag.pdf", extract_images=True)
pages = loader.load()

In [82]:
pages

[Document(metadata={'source': '/content/lora.pdf', 'page': 0}, page_content='LORA: L OW-RANK ADAPTATION OF LARGE LAN-\nGUAGE MODELS\nEdward Hu∗Yelong Shen∗Phillip Wallis Zeyuan Allen-Zhu\nYuanzhi Li Shean Wang Lu Wang Weizhu Chen\nMicrosoft Corporation\n{edwardhu, yeshe, phwallis, zeyuana,\nyuanzhil, swang, luw, wzchen }@microsoft.com\nyuanzhil@andrew.cmu.edu\n(Version 2)\nABSTRACT\nAn important paradigm of natural language processing consists of large-scale pre-\ntraining on general domain data and adaptation to particular tasks or domains. As\nwe pre-train larger models, full ﬁne-tuning, which retrains all model parameters,\nbecomes less feasible. Using GPT-3 175B as an example – deploying indepen-\ndent instances of ﬁne-tuned models, each with 175B parameters, is prohibitively\nexpensive. We propose Low-RankAdaptation, or LoRA, which freezes the pre-\ntrained model weights and injects trainable rank decomposition matrices into each\nlayer of the Transformer architecture, greatly red

In [83]:
# Split text into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents(pages)

In [84]:
docs

[Document(metadata={'source': '/content/lora.pdf', 'page': 0}, page_content='LORA: L OW-RANK ADAPTATION OF LARGE LAN-\nGUAGE MODELS\nEdward Hu∗Yelong Shen∗Phillip Wallis Zeyuan Allen-Zhu\nYuanzhi Li Shean Wang Lu Wang Weizhu Chen\nMicrosoft Corporation\n{edwardhu, yeshe, phwallis, zeyuana,\nyuanzhil, swang, luw, wzchen }@microsoft.com\nyuanzhil@andrew.cmu.edu\n(Version 2)\nABSTRACT\nAn important paradigm of natural language processing consists of large-scale pre-\ntraining on general domain data and adaptation to particular tasks or domains. As\nwe pre-train larger models, full ﬁne-tuning, which retrains all model parameters,\nbecomes less feasible. Using GPT-3 175B as an example – deploying indepen-\ndent instances of ﬁne-tuned models, each with 175B parameters, is prohibitively\nexpensive. We propose Low-RankAdaptation, or LoRA, which freezes the pre-\ntrained model weights and injects trainable rank decomposition matrices into each\nlayer of the Transformer architecture, greatly red

In [85]:
vector_db = Weaviate.from_documents(
    docs, embeddings, client=client, by_text=False
)

In [89]:
print(vector_db.similarity_search("what is lora?", k=3)[0].page_content)

requirements by using a small set of trainable parameters, often termed adapters, while not updating
the full model parameters which remain fixed. Gradients during stochastic gradient descent are
passed through the fixed pretrained model weights to the adapter, which is updated to optimize the
loss function. LoRA augments a linear projection through an additional factorized projection. Given
a projection XW =YwithX∈Rb×h,W∈Rh×oLoRA computes:
Y=XW +sXL 1L2, (3)
whereL1∈Rh×randL2∈Rr×o, and sis a scalar.
Memory Requirement of Parameter-Efficient Finetuning One important point of discussion is
the memory requirement of LoRA during training both in terms of the number and size of adapters
used. Since the memory footprint of LoRA is so minimal, we can use more adapters to improve
performance without significantly increasing the total memory used. While LoRA was designed as a
3


In [90]:
print(vector_db.similarity_search("what is lora?", k=3)[1].page_content)

other words, as we increase the number of trainable parameters3, training LoRA roughly converges
to training the original model, while adapter-based methods converges to an MLP and preﬁx-based
methods to a model that cannot take long input sequences.
No Additional Inference Latency. When deployed in production, we can explicitly compute and
storeW=W0+BA and perform inference as usual. Note that both W0andBA are inRd×k.
When we need to switch to another downstream task, we can recover W0by subtracting BAand
then adding a different B′A′, a quick operation with very little memory overhead. Critically, this
2They represent a negligible number of parameters compared to weights.
3An inevitability when adapting to hard tasks.
4


In [91]:
print(vector_db.similarity_search("what is lora?", k=3)[2].page_content)

LoRA adds trainable pairs of rank decomposition matrices in parallel to existing weight matrices.
As mentioned in Section 4.2, we only apply LoRA to WqandWvin most experiments for simplicity.
The number of trainable parameters is determined by the rank rand the shape of the original weights:
|Θ|= 2×ˆLLoRA×dmodel×r, where ˆLLoRA is the number of weight matrices we apply LoRA to.
6


In [92]:
print(
    vector_db.similarity_search(
        "what is qlora?", k=3)
    )

[Document(metadata={'page': 4, 'source': '/content/qlora.pdf'}, page_content='transfers between the CPU and GPU for error-free GPU processing in the scenario where the GPU\noccasionally runs out-of-memory. The feature works like regular memory paging between CPU RAM\nand the disk. We use this feature to allocate paged memory for the optimizer states which are then\nautomatically evicted to CPU RAM when the GPU runs out-of-memory and paged back into GPU\nmemory when the memory is needed in the optimizer update step.\nQL ORA.Using the components described above, we define QLORAfor a single linear layer in\nthe quantized base model with a single LoRA adapter as follows:\nYBF16=XBF16doubleDequant (cFP32\n1, ck-bit\n2,WNF4) +XBF16LBF16\n1LBF16\n2, (5)\nwhere doubleDequant (·)is defined as:\ndoubleDequant (cFP32\n1, ck-bit\n2,Wk-bit) =dequant (dequant (cFP32\n1, ck-bit\n2),W4bit) =WBF16,(6)\nWe use NF4 for Wand FP8 for c2. We use a blocksize of 64 for Wfor higher quantization precision\nand 

In [93]:

from langchain.prompts import ChatPromptTemplate

template="""You are an assistant for question-answering tasks.
At the start of the question always say :- "here is my answer to the lord"
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""


In [94]:
prompt=ChatPromptTemplate.from_template(template)

In [95]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='You are an assistant for question-answering tasks.\nAt the start of the question always say :- "here is my answer to the lord"\nUse the following pieces of retrieved context to answer the question.\nIf you don\'t know the answer, just say that you don\'t know.\nUse ten sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n'))])

In [96]:
from langchain import HuggingFaceHub

In [97]:
from google.colab import userdata
huggingfacehub_api_token=userdata.get('huggingface_api')

In [98]:
model = HuggingFaceHub(
    huggingfacehub_api_token=huggingfacehub_api_token,
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    model_kwargs={"temperature":1, "max_length":240}
)

In [99]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [100]:
output_parser=StrOutputParser()

In [101]:
retriever=vector_db.as_retriever()

In [108]:
def ensure_complete_sentence(text):
    """Ensures that the output ends with a complete sentence."""
    end_punctuation = ['.', '!', '?']

    # If the text already ends with punctuation, return as is
    if text[-1] in end_punctuation:
        return text

    # Otherwise, find the last occurrence of punctuation and truncate there
    last_punctuation = max(text.rfind(p) for p in end_punctuation)

    if last_punctuation != -1:
        return text[:last_punctuation + 1]

    # If no punctuation is found, return the text as is (shouldn't happen in typical cases)
    return text


In [109]:
from langchain.schema import BaseOutputParser

class PostProcessOutput(BaseOutputParser):
    def parse(self, text):
        return ensure_complete_sentence(text)


In [110]:
post_process_chain = PostProcessOutput()

In [111]:
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
    | post_process_chain
)

In [112]:
print(rag_chain.invoke("what is qlora?"))

Human: You are an assistant for question-answering tasks.
At the start of the question always say :- "here is my answer to the lord"
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: what is qlora?
Context: [Document(metadata={'page': 4, 'source': '/content/qlora.pdf'}, page_content='transfers between the CPU and GPU for error-free GPU processing in the scenario where the GPU\noccasionally runs out-of-memory. The feature works like regular memory paging between CPU RAM\nand the disk. We use this feature to allocate paged memory for the optimizer states which are then\nautomatically evicted to CPU RAM when the GPU runs out-of-memory and paged back into GPU\nmemory when the memory is needed in the optimizer update step.\nQL ORA.Using the components described above, we define QLORAfor a single linear layer in\nthe quantized base model with a sing

In [113]:
print(rag_chain.invoke("How does the RAG model differ from traditional language generation models?"))

Human: You are an assistant for question-answering tasks.
At the start of the question always say :- "here is my answer to the lord"
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: How does the RAG model differ from traditional language generation models?
Context: [Document(metadata={'page': 5, 'source': '/content/rag.pdf'}, page_content='cases for NQ, where an extractive model would score 0%.\n4.2 Abstractive Question Answering\nAs shown in Table 2, RAG-Sequence outperforms BART on Open MS-MARCO NLG by 2.6 Bleu\npoints and 2.6 Rouge-L points. RAG approaches state-of-the-art model performance, which is\nimpressive given that (i) those models access gold passages with speciﬁc information required to\ngenerate the reference answer , (ii) many questions are unanswerable without the gold passages, and\n(iii) not all questions are answerable from